# NN Implementation Flow in TF   
1. **Set hyper-parameters** : learning rate, training epochs, batch size, etc.
2. **Make a data pipelining** : use tf.data
3. **Build a neural network model** : use tf.keras sequential APIs
4. **Define a loss function** : cross entropy
5. **Calculate a gradient** : use tf.GradientTape
6. **Select an optimizer** : Adam optimizer
7. **Define a metric for model's performance** : accuracy
8. **(optional) Make a checkpoint for saving**
9. **Train and Validate a neural network model**

### Import Libraries

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

### Set hyper-parameters

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 현재 작업 디렉토리 가져옴(get current work directory)
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_seq'

# checkpoint 저장용 디렉토리 생성
checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

### Make a data pipelining

In [5]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images,
                                                    train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

### Bulid a NN model - Sequential API

In [6]:
def create_model():
    model = keras.Sequential()
    
    # 첫번째 레이어 추가
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu,
                                  padding='SAME', input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    
    # 두번째 레이어 추가
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu,
                                  padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    
    # 세번째 레이어 추가
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu,
                                  padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    
    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

### Define a loss function

In [15]:
def loss_fn(model, images, labels):
    # Dropout 적용됨(다른 부분에는 영향 X)
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                     labels=labels))
    
    return loss

### Calculate a gradient

In [16]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    
    return tape.gradient(loss, model.variables)

### Select an optimizer / Define a metric for model's performance

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

### Make a checkpoint for saving

In [18]:
checkpoint = tf.train.Checkpoint(cnn=model)

### Train and validate a NN model

In [22]:
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss+loss
        avg_train_acc = avg_train_acc+acc
        train_step += 1
    
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc = avg_test_acc + acc
        test_step += 1
    
    avg_test_acc = avg_test_acc / test_step
    
    print('Epoch:', '{}'.format(epoch+1), 'loss =', '{:.8f}'.format(avg_loss),
          'train accuracy = ', '{:.4f}'.format(avg_train_acc),
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

Epoch: 1 loss = 0.03061724 train accuracy =  0.9935 test accuracy =  0.9928
Epoch: 2 loss = 0.02452373 train accuracy =  0.9954 test accuracy =  0.9882
Epoch: 3 loss = 0.01916130 train accuracy =  0.9965 test accuracy =  0.9916
Epoch: 4 loss = 0.01503806 train accuracy =  0.9974 test accuracy =  0.9932
Epoch: 5 loss = 0.01315721 train accuracy =  0.9979 test accuracy =  0.9918
Epoch: 6 loss = 0.00960220 train accuracy =  0.9984 test accuracy =  0.9935
Epoch: 7 loss = 0.00868033 train accuracy =  0.9988 test accuracy =  0.9914
Epoch: 8 loss = 0.00857024 train accuracy =  0.9986 test accuracy =  0.9919
Epoch: 9 loss = 0.00681301 train accuracy =  0.9991 test accuracy =  0.9935
Epoch: 10 loss = 0.00615207 train accuracy =  0.9992 test accuracy =  0.9928
Epoch: 11 loss = 0.00683468 train accuracy =  0.9990 test accuracy =  0.9924
Epoch: 12 loss = 0.00543248 train accuracy =  0.9994 test accuracy =  0.9928
Epoch: 13 loss = 0.00542313 train accuracy =  0.9994 test accuracy =  0.9929
Epoch: 1